In [ ]:
import PIL
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import torch
import pytorch_lightning as pl

import dataset
import neural_network
import config as cfg

## Parameters

In [ ]:
CLASSES = dataset.metadata.get_classes(cfg.paths.LABELS_CSV["train"])  
CLASSES, CLASSES.size

In [ ]:
IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH = dataset.metadata.get_image_dimensions(cfg.paths.IMG_DIR["train"])
IMG_CHANNELS, IMG_HEIGHT, IMG_WIDTH

## Loading Checkpoint

In [ ]:
SEED = 0
pl.seed_everything(SEED, workers=True)

In [ ]:
data_module = dataset.SkinCancerDataModule(
    cfg.paths.LABELS_CSV,
    cfg.paths.IMG_DIR,
    cfg.hparams.BATCH_SIZE,
    cfg.hparams.DATALOADER_NUM_WORKERS,
    transform=None
)

In [ ]:
model = neural_network.ConvNetwork(CLASSES.size, cfg.hparams.DROPOUT_RATE)
print(model)

In [ ]:
version = 0
CHECKPOINT_DIR = cfg.paths.LOG_DIR / f"lightning_logs/version_{version}" / "checkpoints"

CHECKPOINT_PATHS = list(CHECKPOINT_DIR.glob("*.ckpt")) 
CHECKPOINT_PATHS

In [ ]:
model_module: neural_network.NetworkModule = neural_network.NetworkModule.load_from_checkpoint(
    CHECKPOINT_PATHS[0],
    model=model,
    channels=IMG_CHANNELS,
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    num_classes=CLASSES.size,
    learning_rate=cfg.hparams.LEARNING_RATE
)

## Image

In [ ]:
data_module.setup()
data_module.batch_size = 1 # overwrite with one to get only one image

dataloader = data_module.test_dataloader()
imgs, labels = next(iter(dataloader))

In [ ]:
from torchvision.transforms.functional import to_pil_image

label = labels[0]
img_tensor = imgs[0]

img = to_pil_image(imgs[0].type(torch.uint8))

CLASSES[label]

In [ ]:
plt.imshow(img)
plt.axis("off")

## GradCAM

In [ ]:
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

In [ ]:
target_layers = [model_module.model.third_conv[-1]]

cam = GradCAM(
    model=model_module.model, 
    target_layers=target_layers, 
    use_cuda=True
)

In [ ]:
targets = [ClassifierOutputTarget(label)]

# pass aug_smooth=True and eigen_smooth=True to apply smoothing.
grayscale_cam = cam(input_tensor=imgs, targets=targets)

In [ ]:
norm_rgb_numpy = (img_tensor / 255).numpy()  # it needs to be a float between zero and one
norm_rgb_numpy = np.moveaxis(norm_rgb_numpy, 0, -1)  # it needs to be of shape (height, width, channels)

# grayscale_cam has only one image in the batch
grayscale_cam_first = grayscale_cam[0, :]

visualization = show_cam_on_image(norm_rgb_numpy, grayscale_cam_first, use_rgb=True)

In [ ]:
plt.imshow(visualization)
plt.axis("off")